# meta_pinns算法

## 环境安装

本案例要求 **MindSpore >= 2.0.0** 版本以调用如下接口: *mindspore.jit, mindspore.jit_class, mindspore.data_sink*。具体请查看[MindSpore安装](https://www.mindspore.cn/install)。

此外，你需要安装 **MindFlow >=0.1.0** 版本。如果当前环境还没有安装，请按照下列方式选择后端和版本进行安装。

In [ ]:
mindflow_version = "0.1.0"  # update if needed
# GPU Comment out the following code if you are using NPU.
!pip uninstall -y mindflow-gpu
!pip install mindflow-gpu==$mindflow_version

# NPU Uncomment if needed.
# !pip uninstall -y mindflow-ascend
# !pip install mindflow-ascend==$mindflow_version

## 概述

在不同的物理应用场景下，选取合适的PINNs 损失函数目前仍主要依赖于经验和手工设计。为了解决上述问题，Apostolos F Psarosa等人提出了Meta-PINNs 算法，该算法通过在训练中以梯度下降的方式更新作用于损失函数的超参，从而训练出一组适用于一类同族偏微分方程的超参数。


## 技术路径

求解该问题的具体流程如下：

1. 创建数据集
2. 定义模型和优化器
3. 定义内循环（inner loop）和外循环（outer loop）的前向传播函数和梯度函数
4. 定义内循环和外循环的训练步骤
5. 设置训练参数，如学习率、迭代次数
6. 进行训练，并在每个外循环间隔上进行模型评估

## 引入代码包

In [ ]:
import argparse
import os
import time

import numpy as np

from mindspore import context, nn, get_seed, set_seed, data_sink

from mindflow.cell import MultiScaleFCSequential
from mindflow.utils import load_yaml_config

下述`src`包可以在[research/meta_pinns/src](https://gitee.com/mindspore/mindscience/tree/master/MindFlow/applications/research/meta_pinns/src)下载。

In [ ]:
from src import create_train_dataset, create_problem, create_trainer, create_normal_params
from src import re_initialize_model, evaluate, plot_l2_error, plot_l2_comparison_error
from src import WorkspaceConfig, TrainerInfo

set_seed(0)
np.random.seed(0)

进行参数配置，其中--case可选"burgers", "l_burgers", "cylinder_flow", "periodic_hill"

In [ ]:
parser = argparse.ArgumentParser(description="meta-pinns")
parser.add_argument("--case", type=str, default="burgers", choices=["burgers", "l_burgers", "periodic_hill", "cylinder_flow"],
                    help="choose burgers")
parser.add_argument("--mode", type=str, default="GRAPH", choices=["GRAPH", "PYNATIVE"],
                    help="Running in GRAPH_MODE OR PYNATIVE_MODE")
parser.add_argument("--device_target", type=str, default="Ascend", choices=["GPU", "Ascend"],
                    help="The target device to run, support 'Ascend', 'GPU'")
parser.add_argument("--device_id", type=int, default=0,
                    help="ID of the target device")
parser.add_argument("--config_file_path", type=str,
                    default="./configs/burgers.yaml")
input_args = parser.parse_args()

context.set_context(mode=context.GRAPH_MODE if input_args.mode.upper().startswith("GRAPH")
                    else context.PYNATIVE_MODE,
                    save_graphs=input_args.save_graphs,
                    save_graphs_path=input_args.save_graphs_path,
                    device_target=input_args.device_target,
                    device_id=input_args.device_id)
print(
    f"Running in {input_args.mode.upper()} mode, using device id: {input_args.device_id}.")
use_ascend = context.get_context(attr_key='device_target') == "Ascend"
print(use_ascend)
print("pid:", os.getpid())

确定要训练的方程的、进行yaml文件的加载

In [ ]:
# load configurations
case_name = input_args.case
config = load_yaml_config(input_args.config_file_path)
model_config = config["model"]
test_config = config["meta_test"]
summary_config = config["summary"]
lamda_config = config["lamda"]
meta_train_config = config["meta_train"]
initial_lr = config["optimizer"]["initial_lr"]

## 创建数据集

在案例训练中未使用方程真实值，仅对方程内部点、边界点、初始点进行采样即可。

inner_train_dataset用于内部循环训练，outer_train_dataset用于外部循环训练。

In [ ]:
# create dataset
inner_train_dataset = create_train_dataset(
    case_name, config, get_seed() + 1)
outer_train_dataset = create_train_dataset(
    case_name, config, get_seed() + 2)

## 构建模型

本例使用简单的全连接网络，网络形状由yaml文件决定。

In [ ]:
# define models and optimizers
model = MultiScaleFCSequential(in_channels=config["model"]["in_channels"],
                               out_channels=config["model"]["out_channels"],
                               layers=config["model"]["layers"],
                               neurons=config["model"]["neurons"],
                               residual=config["model"]["residual"],
                               act=config["model"]["activation"],
                               num_scales=1)

## 模型训练

使用**MindSpore >= 2.0.0**的版本，可以使用函数式编程范式训练神经网络。

此处采用元学习算法学习PINN训练过程中损失函数中项的权重。

并在每个外循环间隔上在未见过的方程上进行模型评估。

In [ ]:
lamda = lamda_config["initial_lamda"]
problem = create_problem(lamda, case_name, model, config)
inner_optimizer = nn.SGD(model.trainable_params(),
                         initial_lr)
outer_optimizer = nn.Adam(problem.get_params(),
                          initial_lr)

if use_ascend:
    from mindspore.amp import DynamicLossScaler, auto_mixed_precision
    loss_scaler = DynamicLossScaler(1024, 2, 100)
    auto_mixed_precision(model, 'O1')
else:
    loss_scaler = None

inner_trainer = create_trainer(TrainerInfo(case_name, model, inner_optimizer, problem,
                                           use_ascend, loss_scaler, config, False, True))
outer_trainer = create_trainer(TrainerInfo(case_name, model, outer_optimizer,
                                           problem, use_ascend, loss_scaler, config, True, True))

inner_train_step = inner_trainer.train_step
outer_train_step = outer_trainer.train_step

iteration_str = "iterations"
inner_iters = meta_train_config["inner_loop"][iteration_str]
outer_iters = meta_train_config["outer_loop"][iteration_str]

steps_per_epochs = inner_train_dataset.get_dataset_size()
inner_sink_process = data_sink(
    inner_train_step, inner_train_dataset, sink_size=1)
outer_sink_process = data_sink(
    outer_train_step, outer_train_dataset, sink_size=1)

lamda_min = lamda_config["lamda_min"]
lamda_max = lamda_config["lamda_max"]

used_lamda = [lamda_config["eva_lamda"]]
best_params = problem.get_params()
best_l2 = 1e10

# starting meta training
eva_l2_errors = []
for epoch in range(1, 1 + outer_iters):
    # train
    lamda = np.random.uniform(lamda_min, lamda_max)
    if lamda not in used_lamda:
        used_lamda.append(lamda)
    time_beg = time.time()
    model.set_train(True)

    if epoch % meta_train_config["reinit_lamda"] == 0:
        problem.lamda = lamda
    if epoch % meta_train_config["reinit_epoch"] == 0:
        re_initialize_model(model, epoch)

    for _ in range(1, 1 + inner_iters):
        for _ in range(steps_per_epochs):
            inner_sink_process()
    for _ in range(steps_per_epochs):
        cur_loss = outer_sink_process()

    print("epoch: %s loss: %s epoch time: %.3fms",
          epoch, cur_loss, (time.time() - time_beg) * 1000)

    if epoch % meta_train_config["eva_interval_outer"] == 0:
        # evaluate current model on unseen lamda
        print("learned params are: %s",
              problem.get_params(if_value=True))
        eva_iter = meta_train_config["eva_loop"][iteration_str]
        eva_l2_error = evaluate(WorkspaceConfig(epoch, case_name, config, problem.get_params(), eva_iter,
                                                eva_iter, use_ascend, loss_scaler, False, True, False, None))
        eva_l2_errors.append(eva_l2_error[0])
        if eva_l2_error[0] < best_l2:
            best_l2 = eva_l2_error[0]
            best_params = problem.get_params()

print(best_l2)
for param in best_params:
    print(param.asnumpy())

plot_l2_error(case_name, summary_config["visual_dir"],
              meta_train_config["eva_interval_outer"], eva_l2_errors)

# start comparing
test_iter = test_config[iteration_str]
test_interval = test_config["cal_l2_interval"]

# start meta training
meta_l2_errors = evaluate(WorkspaceConfig(None, case_name, config, best_params,
                                          test_iter, test_interval,
                                          use_ascend, loss_scaler, False, True,
                                          True, f"{case_name}_meta_testing"))
# end meta training

# start normal training
normal_params = create_normal_params(case_name)

normal_l2_errors = evaluate(WorkspaceConfig(None, case_name, config, normal_params,
                                            test_iter, test_interval,
                                            use_ascend, loss_scaler, False, False,
                                            True, f"{case_name}_normal_training"))
# end normal training

plot_l2_comparison_error(
    case_name, summary_config["visual_dir"], test_interval, meta_l2_errors, normal_l2_errors)

epoch: 1 loss: [0.86779684] epoch time: 23640.816ms
epoch: 2 loss: [0.9413994] epoch time: 1189.008ms
epoch: 3 loss: [0.9189246] epoch time: 1087.861ms
epoch: 4 loss: [0.9275309] epoch time: 1123.993ms
...
epoch: 50 l2_error: 1.0040625699565209 epoch time: 8788.293ms
...
epoch: 1000 loss: [0.9453384] epoch time: 1885.815ms
learned params are: pde weight=[1.865994] ic weight=[1.8658787] bc_weight=[1.8648634]
...
-------------------------------------start meta testing-------------------------------------
epoch: 0 l2_error: 1.0029983755797225 epoch time: 12976.881ms
epoch: 100 l2_error: 0.8387432039920448 epoch time: 111.595ms
epoch: 200 l2_error: 0.5762822379368656 epoch time: 111.311ms
...
epoch: 19800 l2_error: 0.033477298861820166 epoch time: 76.523ms
epoch: 19900 l2_error: 0.027350145606692883 epoch time: 79.059ms
-------------------------------------end meta testing-------------------------------------
-------------------------------------start normal training-----------------------

![burgers_l2](./images/burgers_l2.png)